<a href="https://colab.research.google.com/github/aSafarpoor/OSN_FAD/blob/main/ADA/Adversaries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#initialization

In [70]:
SEED = 1

In [71]:
import numpy as np
import random
from sklearn.metrics import confusion_matrix, accuracy_score
import networkx as nx
from tqdm import tqdm

In [72]:
def load_txt_file(filename):
	try:
		with open(filename, 'r') as file:
			return [int(line.strip()) for line in file]
	except:
		with open(filename, 'r') as file:
			first_line = file.readline().strip()  # Read only the first line
			return [int(num) for num in first_line.split()]

def load_txt_file_for_edges(filename):
	c = 0
	with open(filename, 'r') as file:
		return [list(map(int,line.strip().split())) for line in file]

In [73]:
def partitioner(x,ratio,do_shuffle=True):
    midpoint = int(len(x) * ratio)

    # Split the list into two parts
    if do_shuffle:
        random.seed(SEED)
        random.shuffle(x)
    return x[:midpoint], x[midpoint:]


In [74]:
edges = load_txt_file_for_edges('edges.txt')

In [75]:
bknown = load_txt_file('btrain.txt')
sknown = load_txt_file('strain.txt')


btest = load_txt_file('btest.txt')
stest = load_txt_file('stest.txt')

nodes = list(set(np.array(edges).reshape(-1)))
num_nodes = len(nodes)

print(len(bknown),len(sknown),len(btest),len(stest))

80 80 323 323


# Data Generation

In [76]:
def remap_graph_and_update_lists(g,nodes_in_order, list_of_lists):
    # This function rename nodes to have n nodes with names {0,1,...,n-1}
    # and update edges based on that

    nodes = nodes_in_order
    node_mapping = {old_node: new_index for new_index, old_node in enumerate(nodes)}

    g_mapped = nx.relabel_nodes(g, node_mapping)

    outlists = []
    for list_i in list_of_lists:
        temp = [node_mapping[node] for node in list_i if node in node_mapping]
        outlists.append(temp[:])

    return g_mapped, outlists

We will have:
+ bknown_test(part of trainset moved for known in test),btrain,btest(labels)
+ sknown_test(part of trainset moved for known in test),strain,stest(labels)
+ unknowntrain,unknowntest
+ target (part of sybils, can be in test or train)
+ trainX,testX,trainY,testY
+ train_edges,test_edges

we seperated main btrain and strain to know nodes for train and test.

It is generally for inductive learning. For transductive you can add xknown_test back to ktrain


In [77]:
def sample_targets(stest,sknown,ratio):

    sybils = list(set(stest+sknown))

    random.seed(SEED)
    targets = random.sample(sybils[:], int(len(sybils)*ratio))

    return targets

In [78]:
def partitioner_transductive_inductive(bknown,sknow,btest,stest,edges,nodes,ratio,is_transductive):
    btrain,bknown_test = partitioner(bknown, ratio = ratio)
    strain,sknown_test = partitioner(sknown, ratio = ratio)

    knownset = set(bknown+sknown)
    unknownset = set(nodes)- knownset

    unknowntrain,unknowntest = partitioner(list(unknownset)[:], ratio = ratio)
    del(unknownset)

    trainX = btrain+strain+unknowntrain
    trainY = [0 for i in btrain]+[1 for i in strain]+[[-1 for i in unknowntrain]]
    testX = btest+stest+unknowntest+bknown_test+sknown_test+trainX
    textCompleteY = [0 for i in btest]+[1 for i in stest]+[-1 for i in unknowntest]+[0 for i in bknown_test]+[1 for i in sknown_test]+trainY
    textInitY = [-1 for i in btest]+[-1 for i in stest]+[-1 for i in unknowntest]+[0 for i in bknown_test]+[1 for i in sknown_test]+trainY

    if is_transductive:
        return (bknown_test,
                btrain,
                btest,
                sknown_test,
                strain,
                stest,
                unknowntrain,
                unknowntest,
                bknown_test,
                sknown_test,
                edges,
                trainX,
                trainY,
                testX,
                textCompleteY,
                textInitY
                )

    else:
        g = nx.Graph()
        g.add_edges_from(edges)


        gtrain = g.subgraph(trainX)
        gtest = g.subgraph(testX)

        gtrain, [btrain,strain,unknowntrain] = remap_graph_and_update_lists(g=gtrain,
                                                                            nodes_in_order = trainX,
                                                                            list_of_lists = [btrain,strain,unknowntrain])

        gtest, [btest,stest,unknowntest,bknown_test,sknown_test] = remap_graph_and_update_lists(g=gtest,
                                                                                                nodes_in_order=testX,
                                                                                                list_of_lists=[btest,stest,unknowntest,bknown_test,sknown_test])

        train_edges = list(gtrain.edges())
        test_edges = list(gtest.edges())

        return (bknown_test,
                btrain,
                btest,
                sknown_test,
                strain,
                stest,
                unknowntrain,
                unknowntest,
                bknown_test,
                sknown_test,
                train_edges,
                test_edges,
                trainX,
                trainY,
                testX,
                textCompleteY,
                textInitY
                )


# Adversarial Attacks

## Changing Edges Random

In [122]:
def changing_edges_random(g,allnodes,targets,addbudget,removebudget):
    random.seed(SEED)
    G = g.copy()

    es = list(G.edges())

    new_es = []

    edge_dictionary = {}
    for t in targets:
         edge_dictionary[t] = [v for u, v in G.out_edges(t)]

    c = 0
    while c < addbudget:
        x,y = random.sample(targets, 2)
        if G.has_edge(x, y):
            pass
        else:
            G.add_edge(x,y)
            c+=1
            new_es.append([x,y])

    c = 0


    while c < removebudget:
        x = random.sample(targets,1)[0]

        y = random.sample(edge_dictionary[x],1)[0]
        try:
            G.remove_edge(x,y)
            c+=1
        except:
            pass

    return G,list(G.edges())



In [123]:
g = nx.DiGraph()
g.add_edges_from(edges)

random.seed(SEED)
targets = sample_targets(stest,sknown,ratio=0.1)
G,es = changing_edges_random(g,nodes,targets,addbudget=100,removebudget=100)


In [124]:
(
    bknown_test,
    btrain,
    btest,
    sknown_test,
    strain,
    stest,
    unknowntrain,
    unknowntest,
    bknown_test,
    sknown_test,
    edges,
    trainX,
    trainY,
    testX,
    textCompleteY,
    textInitY
) = partitioner_transductive_inductive(bknown[:], sknown[:], btest[:], stest[:], es[:], nodes[:], ratio=0.3, is_transductive=True)


(
    bknown_test,
    btrain,
    btest,
    sknown_test,
    strain,
    stest,
    unknowntrain,
    unknowntest,
    bknown_test,
    sknown_test,
    train_edges,
    test_edges,
    trainX,
    trainY,
    testX,
    textCompleteY,
    textInitY
) = partitioner_transductive_inductive(bknown[:], sknown[:], btest[:], stest[:], es[:], nodes[:], ratio=0.3, is_transductive=False)